In [ ]:
# Map practice from the gmaps documentation
# URL https://jupyter-gmaps.readthedocs.io/en/v0.3.2/gmaps.html

# Import the dependencies.
import pandas as pd

import numpy as np
import random

# very important to include this module - this is what was causing error
import requests

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Import the datetime module from the datetime library.
from datetime import datetime

# check do we have the config.py set up correctly and linked into this notebook? 
from config import weather_api_key

# still need to install gmaps in the back end using anaconda prompt- this might be a good office hours question
# it was still waiting for a y/n answer so it had not installed yet
import gmaps
import gmaps.datasets

import requests
# Import the API key.
from config import g_key

# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)


In [ ]:
# generate some data - this is an example, I will make my own 
# why does it open the map in Geneva as the default? 

# This is from the gmaps getting started documentation
# I thought if I could get THIS to work then the other heat map would work too
# Henry (TA) Helped me fix this on July 27- I still needed to enable two modules in anaconda
# jupyter nbextension enable --py gmaps
# jupyter nbextension enable --py widgetsnbextension


earthquake_df = gmaps.datasets.load_dataset_as_df('earthquakes')
earthquake_df.head()

In [ ]:
locations = earthquake_df[['latitude', 'longitude']]
weights = earthquake_df['magnitude']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
# fig

earthquake_df

In [ ]:
# first test my city to pick some USA cities and get the data for them 
# my_city = citipy.nearest_city(40.1, -96.2).city_name
my_x = 39.117
my_y = -94.4

my_city = citipy.nearest_city(my_x,my_y).city_name
my_country = citipy.nearest_city(my_x,my_y).country_code
print(my_city, my_country)

In [ ]:
# make one more alternate version of the lats list - in equal increments but "reflecting off the wall"
# for the increment pick something that is not close to a multiple of whole numbers like the quotient of two primes
# I will generate 200 values on both x and y coords that you can choose to use or not use
lattice_x = []
lattice_y = []
# starting lat/lng is geographic center of CONUS
last_lat = 37.27
last_lng = -93.32
# as increments use a ration of prime numbers so the sequence will not repeat
lat_increment = round(17/13 ,2)
lng_increment = round(61/23 ,2)
for i in range(200):
    # percent sign is modulo operator
    new_lat = 24 + round((last_lat + lat_increment) % 24,2)
    new_lng = -123 + round((last_lng + lng_increment) % 54,2)

    lattice_x.append(new_lat)
    lattice_y.append(new_lng)

    last_lat = round(new_lat,2)
    last_lng = round(new_lng,2)

    # how do we round to 2 decimal places
# will this terminate    

In [ ]:
# the lattice will scroll through the range of lats multiple times in sequence but with no repeats
# the lng values can still be random unless I still am not satisfied
len(lattice_x)


In [ ]:

my_lats = np.random.uniform(low=24.000, high=49.000, size=200)
# US lng ranes from -124 WA to -66.0 Maine
my_lngs = np.random.uniform(low=-125.000, high=-65.000, size=201)
# my_lngs
# random doesn't really give me the coverage I want but lets see the map before we decide
lattice_y

In [ ]:
my_lats

In [ ]:
# The pin drops are all banding into the same range of lats/longs so it is still not really a random sample
# some states got multiple hits and some got none at all - see pin drop map in last frame
# maybe the only way to really cover all 50 states is to do a lattice

counter = 0

# change my_lats to lattice_x for testing
for i in range(200):
    print(lattice_x[i], lattice_y[i])
    
    counter +=1
# nex t - making a bunch of citypy calls   



In [ ]:
# use the new lattice_x for this test to replace my_lats
my_zip = zip(lattice_x, lattice_y)
my_new_list = list(my_zip)
len(my_new_list)
# my_new_list[:10]

In [ ]:
# now I have a DF of exactly 10 GPS coords
new_df = pd.DataFrame(my_new_list[:150],columns=['Lat', 'Lng'])
new_df.head(20)


In [ ]:
# Create a list for holding the cities.
my_cities = []
all_my_cities = []
# Identify the nearest city for each latitude and longitude combination.
# substitute lattic_x for my_lats
for lat in lattice_x:
    lat_shift = 2.2
    lng_shift = 0.5
    # using the lattice_x method we will not need the SHIFT any more
    lng = my_lngs[counter]
    my_city = citipy.nearest_city(lat+lat_shift, lng+lng_shift).city_name

    # get a list of all the cities for testing to see which city had the most hits
    all_my_cities.append(my_city)

    # If the city is unique, then we will add it to the cities list.
    if my_city not in my_cities:
        my_cities.append(my_city)

# Print the city count to confirm sufficient count.
# len(all_my_cities)
all_my_cities

# with 200 random points we only got 32 unique places - that seems odd
# also I would like to select only cities above a certain population, not just any city with over 500 people
# am I supposed to believe that for every pin dropped in the gulf of Mexico, 'santander jimenez' is the only nearest city?
# its hard to believe - I am going to test citypy more to see if I give it the exact coords of a city what does it return
# also I am going to randomize the pin points even further by adding a small increment to the lat value
# with a lattice for both x and y we get lompoc and san quintin a lot but i think we get more variety

In [ ]:
# this is crazy that my new method results in fewer cities and more duplicates - how is that possible ?
# maybe I have to see the pins on the map to realize what happened- where is hamilton anyway and why so many?
# with lattice-y included we get seattle and tacoma but the list of cities is still shorter than I hoped
my_cities

In [ ]:
len(my_cities)

In [ ]:
# finish set up for the map layer 
try_weights = [10,10,10,10,10,10,10,10,10,10]

# this should give me a list of all 10s the same lenth as the number of lats
my_weights = []
for lat in my_lats:
        my_weights.append(20)
           
# can I use 10 random points to make my own map layer ?
len(my_weights)

In [ ]:
# only run this last cell when you are ready - these are the pin drops before we link them to a city
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(new_df, weights=my_weights[:150])
fig.add_layer(heat_layer)
fig

# fig.add_layer(gmaps.heatmap_layer(my_new_df, weights=my_weights[:150]))
# fig
# This is the map of just the pins we are dropping on the map, not the nearest city
# Do I get charged $$ every time I generate a new map ? ? ? ?